# 🔤 Tokenizer Training: Building a BPE Tokenizer

Before we can train our SLM, we need to convert text into tokens. This notebook trains a **Byte Pair Encoding (BPE)** tokenizer on our pre-1986 corpus.

**Why tokenization matters:** A poorly designed tokenizer fragments technical terms and equations, making it harder for the model to learn. We want tokens that respect word boundaries and technical vocabulary.

---
## 1. Setup

We'll use the Hugging Face `tokenizers` library - it's fast and flexible.

In [ ]:
# You might need: pip install tokenizers
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.processors import TemplateProcessing
from pathlib import Path

DATA_DIR = Path("../data/pre1986_training_streams_v1_FINAL")
TOKENIZER_DIR = Path("../tokenizer")
TOKENIZER_DIR.mkdir(exist_ok=True)

print("✓ Imports successful")

---
## 2. How BPE Works (Quick Overview)

**Byte Pair Encoding** builds a vocabulary by:
1. Starting with individual characters
2. Finding the most frequent pair of adjacent tokens
3. Merging that pair into a new token
4. Repeating until we hit our target vocabulary size

For example:
```
"low" "lower" "newest" "widest"
→ After BPE: "low" "low" "er" "new" "est" "wid" "est"
→ Common subwords like "est" become their own tokens
```

This is great for handling rare technical terms - even if we've never seen "thermocouple" before, we might have tokens for "thermo" and "couple".

---
## 3. Configure the Tokenizer

Key decisions:
- **Vocab size: 32,000** - Large enough for technical vocabulary, small enough to keep embeddings manageable
- **Special tokens:** We need `<EOS>`, `<PAD>`, and `<UNK>` at minimum

In [ ]:
# Initialize a blank BPE tokenizer
tokenizer = Tokenizer(BPE(unk_token="<UNK>"))

# Pre-tokenization: split on whitespace first
# This prevents weird merges across word boundaries
tokenizer.pre_tokenizer = Whitespace()

# Configure the trainer
trainer = BpeTrainer(
    vocab_size=32000,
    min_frequency=2,  # Token must appear at least twice
    special_tokens=[
        "<PAD>",   # Padding for batching
        "<UNK>",   # Unknown tokens
        "<EOS>",   # End of sequence (document boundary)
        "<BOS>",   # Beginning of sequence (optional, but nice to have)
    ],
    show_progress=True
)

print("Tokenizer configured with:")
print(f"  - Target vocab size: 32,000")
print(f"  - Special tokens: <PAD>, <UNK>, <EOS>, <BOS>")

---
## 4. Train the Tokenizer

**Important:** We train ONLY on the base pretraining corpus, not the fine-tuning data.

Why? The tokenizer should represent general language patterns. Fine-tuning data is for shaping *reasoning*, not vocabulary.

In [ ]:
# Training on base_stream only
training_files = [str(DATA_DIR / "base_stream.txt")]

print(f"Training tokenizer on: {training_files}")
print("This might take a minute...\n")

# Train!
tokenizer.train(training_files, trainer)

print(f"\n✓ Training complete!")
print(f"  Final vocabulary size: {tokenizer.get_vocab_size():,}")

---
## 5. Test the Tokenizer

Let's see how it handles different types of text.

In [ ]:
def show_tokenization(text):
    """Display how text gets tokenized."""
    encoding = tokenizer.encode(text)
    print(f"Input:  '{text}'")
    print(f"Tokens: {encoding.tokens}")
    print(f"IDs:    {encoding.ids}")
    print(f"Count:  {len(encoding.tokens)} tokens")
    print()

# Test on various inputs
print("=== Regular English ===")
show_tokenization("The quick brown fox jumps over the lazy dog.")

print("=== Technical Text ===")
show_tokenization("The thermocouple measures temperature differentials.")

print("=== Equations ===")
show_tokenization("E = mc^2 where m is mass and c is the speed of light.")

print("=== Control Systems ===")
show_tokenization("The PID controller adjusts the feedback loop gain.")

---
## 6. Vocabulary Analysis

Let's peek at what tokens we learned.

In [ ]:
vocab = tokenizer.get_vocab()

# Sort by token ID to see the order they were added
sorted_vocab = sorted(vocab.items(), key=lambda x: x[1])

print("First 20 tokens (special + single chars):")
for token, idx in sorted_vocab[:20]:
    print(f"  {idx:5d}: '{token}'")

print("\n...\n")

print("Sample of learned merge tokens (around ID 5000):")
for token, idx in sorted_vocab[5000:5020]:
    print(f"  {idx:5d}: '{token}'")

In [ ]:
# Look for technical terms we'd want as single tokens
technical_terms = ['system', 'control', 'reactor', 'pressure', 'temperature',
                   'feedback', 'stability', 'equation', 'function', 'transfer']

print("Technical terms in vocabulary:")
for term in technical_terms:
    if term in vocab:
        print(f"  ✓ '{term}' is a single token (ID: {vocab[term]})")
    else:
        # Show how it gets split
        tokens = tokenizer.encode(term).tokens
        print(f"  ✗ '{term}' splits into: {tokens}")

---
## 7. Save the Tokenizer

We'll save this and use it for all training phases. **Never retrain the tokenizer** - it needs to stay consistent.

In [ ]:
# Save to disk
tokenizer_path = TOKENIZER_DIR / "tokenizer.json"
tokenizer.save(str(tokenizer_path))

print(f"✓ Tokenizer saved to: {tokenizer_path.resolve()}")

# Verify we can load it back
loaded_tokenizer = Tokenizer.from_file(str(tokenizer_path))
test = loaded_tokenizer.encode("This is a test.")
print(f"✓ Reload test passed: {test.tokens}")

---
## 8. Tokenization Statistics

Finally, let's see how efficiently our tokenizer compresses the training data.

In [ ]:
# Load a sample of the base stream
with open(DATA_DIR / "base_stream.txt", 'r', encoding='utf-8') as f:
    sample = f.read(100000)  # First 100KB

encoding = tokenizer.encode(sample)

chars = len(sample)
tokens = len(encoding.tokens)
ratio = chars / tokens

print("Tokenization efficiency (100KB sample):")
print(f"  Characters: {chars:,}")
print(f"  Tokens: {tokens:,}")
print(f"  Chars per token: {ratio:.2f}")
print(f"\n  (Higher is better - fewer tokens for same text)")

---
## Summary

We now have a trained BPE tokenizer with:
- **32K vocabulary** learned from pre-1986 text
- **Special tokens:** `<PAD>`, `<UNK>`, `<EOS>`, `<BOS>`
- Saved to `tokenizer/tokenizer.json`

**Next:** In notebook 03, we'll build the model architecture using this tokenizer.